# Quick MNIST

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn,optim,tensor
from torch.utils.data import TensorDataset, DataLoader
import pickle, gzip
import numpy as np
import random


SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        # Entrenamiento
        model.train()
        for xb,yb in train_dl: 
            pb = model(xb)
            loss = loss_func(pb,yb)  
            loss.backward()
            opt.step()
            opt.zero_grad()
        # Validación
        model.eval()
        losses = list()
        nums = list()
        with torch.no_grad():
            for xb,yb in valid_dl:
                pb = model(xb)
                loss  = loss_func(pb,yb)
                num = len(xb)
                losses.append(loss.item())
                nums.append(num)
        val_loss = np.sum(np.multiply(losses,nums)) / np.sum(nums)
        print(epoch, val_loss)


class Convert_x():
    def __init__(self, dl):
        self.dl = dl
    def __len__(self): return len(self.dl)
    def __iter__(self):
        # batches = iter(self.dl)
        for b in self.dl:
            xp = b[0].view(-1,1,28,28).to(dev)
            yp = b[1].to(dev)
            yield xp, yp
            


def get_dataloader(x,y,bs,shuffle):
    ds = TensorDataset(*map(tensor, (x,y)))
    dl = DataLoader(ds, batch_size=bs, shuffle=shuffle)
    dlp = Convert_x(dl)
    return dlp

class Aplanar(nn.Module):
    def __init__(self):
        super().__init__() # Llamada al constructor de la clase padre
    def forward(self, x):
        return x.view(x.size(0),-1)


bs=64
lr=0.1
epochs=4

with gzip.open('data/mnist.pkl.gz', 'rb') as f:
    ((train_x, train_y), (valid_x, valid_y), _) = pickle.load(f, encoding='latin-1')

model = nn.Sequential(
    nn.Conv2d(1,  16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
    Aplanar()
).to(dev)
loss_func = F.cross_entropy
opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

train_dl = get_dataloader(train_x, train_y, bs,   shuffle=False)
valid_dl = get_dataloader(valid_x, valid_y, bs*2, shuffle=True )
fit(epochs, model, loss_func, opt, train_dl, valid_dl)

0 0.4270922985076904
1 0.2640626178264618
2 0.22213245170116425
3 0.19952659080028534
